In [42]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [43]:
#Model definition
class Model(nn.Module):
    def __init__(self, output_features=1, input_features=2, l1=9, l2=10):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.out1 = nn.Linear(l2, output_features)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.out1(x)
        return x

In [47]:
#Load data
githuburl = "https://github.com/nikojlieto/antennann/raw/main/data/5tags-circle-antennasweep%20(calculations%20included).csv"
my_df = pd.read_csv(githuburl)

In [48]:
list(my_df.columns)

['id',
 'absolute_timestamp',
 'relative_timestamp',
 'interrogator_timestamp',
 'freeform',
 'rssi',
 'epc96',
 'doppler',
 'phase',
 'antenna',
 'rospecid',
 'channelindex',
 'tagseencount',
 'accessspecid',
 'inventoryparameterspecid',
 'lastseentimestamp',
 'antennastate',
 'radial distance (m)',
 'time (seconds)',
 'doppler ()',
 'interrogation frequency (Hz)',
 'phase (radians)',
 'prxval ()',
 'prx deoscillated moving parts ()',
 'prx moving parts ()',
 'prx linear ()',
 'wavelength ()',
 'capital R',
 'interrogation frequency (MHz)',
 'capital R (from MHz)',
 'New R from Prx',
 'New R from RSSI']

In [46]:
#Data preparation
input_columns = ['phase (radians)', 'prxval ()']
X = my_df[input_columns].values
output_columns = ['New R from Prx']
y = my_df[output_columns].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

KeyError: "None of [Index(['New R from Prx'], dtype='object')] are in the [columns]"

In [ ]:
#Model instantiation
model = Model(output_features=1, input_features=len(input_columns))
criterion = nn.MSELoss()  # Changed to MSE for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#Training loop
epochs = 100
losses = []
for i in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.item())
    if i % 10 == 0:
        print(y_pred)
        print(f'Epoch: {i} and loss: {loss.item()}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


tensor([[1200.3059],
        [1144.0338],
        [1449.8485],
        ...,
        [1617.0361],
        [3075.1956],
        [1283.2811]], grad_fn=<AddmmBackward0>)
Epoch: 0 and loss: 2.896399786988976e+30
tensor([[1694.6335],
        [1614.9143],
        [2046.9689],
        ...,
        [2282.9539],
        [4338.0298],
        [1811.4238]], grad_fn=<AddmmBackward0>)
Epoch: 10 and loss: 2.896399786988976e+30
tensor([[2320.5620],
        [2211.0669],
        [2803.0759],
        ...,
        [3126.1650],
        [5936.0396],
        [2480.0649]], grad_fn=<AddmmBackward0>)
Epoch: 20 and loss: 2.896399786988976e+30
tensor([[3095.8572],
        [2949.3894],
        [3739.6375],
        ...,
        [4170.6079],
        [7914.3257],
        [3308.1548]], grad_fn=<AddmmBackward0>)
Epoch: 30 and loss: 2.896399786988976e+30
tensor([[ 4038.3813],
        [ 3846.8713],
        [ 4878.2358],
        ...,
        [ 5440.3496],
        [10318.2295],
        [ 4314.7471]], grad_fn=<AddmmBackward0

In [ ]:
#Model evaluation
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_eval = model(X_test)
    loss = criterion(y_eval, y_test)
    print(f'Test Loss: {loss.item()}')

Test Loss: 2.911350270368695e+30


In [ ]:
#Save the trained model
torch.save(model.state_dict(), 'trained_model.pt')